In [0]:
import tensorflow as tf
import os
import sys

In [0]:
os.chdir('/content')

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
from google.colab import auth
auth.authenticate_user()

In [5]:

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -debug -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Starting application setup (label=default, base_dir=).
[0.000096] TID=0: Loading configuration from /root/.gdfuse/default/config...done
Opening log file: /root/.gdfuse/default/gdfuse.log
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [6]:
!mkdir -p godrive
!google-drive-ocamlfuse godrive

!ls godrive

Colab Notebooks  Ionas_Pakalnishkis_Eng.docx	  object_det
diplom		 Ionas_Pakalnishkis_Eng.docx.odt


In [7]:

!pip3 install keras

In [20]:
os.chdir('..')
!ls

negative  positive  train  validation


## Split positive, negative to train/positive, train/negation, validation/positive, validation/negative, and resize images to one size

In [31]:
import numpy as np
import shutil
import cv2


def copy_images_from_folder(current_directory, values, token):
  split_images = {'train/': (0, int(0.7 * len(values))), 'validation/': (int(0.7 * len(values)) + 1, len(values))}
  
  print('Copying files from %s' % current_directory)
  for directory in split_images.keys():
    
    dest_directory = directory + current_directory
    print('Copying and resize images to %s' % dest_directory)
    
    
    for image_path in values[split_images[directory][0] : split_images[directory][1]]:
      
      image = cv2.imread(current_directory + image_path)
      resized_image = cv2.resize(image, (50, 50)) 
      cv2.imwrite('%sresult%d.png' % (dest_directory, token), resized_image)
      token += 1
      if (token % 10 == 0):
        print(token)
   
  return token
    
    

token = 0
for folder in ['positive/', 'negative/']:
  values = os.listdir(folder)
  token = copy_images_from_folder(folder, values, token)


Copying files from positive/
Copying and resize images to train/positive/
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
Copying and resize images to validation/positive/
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
Copying files from negative/
Copying and resize images to train/negative/
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820


In [35]:
!ls
print (len(os.listdir('train/positive')))
print (len(os.listdir('train/negative')))
print (len(os.listdir('validation/positive')))
print (len(os.listdir('validation/negative')))

negative  positive  train  validation
948
1318
399
564


In [41]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks



epochs = 10
train_data_dir = './train'
validation_data_dir = './validation'

img_width, img_height = 50, 50
nb_train_samples = 2260
nb_validation_samples = 960
nb_filters1 = 8
nb_filters2 = 16
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 2
batch_size = 32
lr = 0.0004

model = Sequential()
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Conv2D(nb_filters2, (conv2_size, conv2_size), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_first'))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
  )

test_datagen = ImageDataGenerator(
    rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

"""
Tensorboard log
"""
# log_dir = './tf-log/'
# tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
# cbks = [tb_cb]

model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    #callbacks=cbks,
    validation_steps=nb_validation_samples)

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

Found 2266 images belonging to 2 classes.
Found 963 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., validation_steps=960, steps_per_epoch=70)`


Epoch 1/10
70/70 [==============================] - 184s 3s/step - loss: 0.4941 - acc: 0.7488 - val_loss: 0.3593 - val_acc: 0.8110
Epoch 2/10
70/70 [==============================] - 155s 2s/step - loss: 0.3279 - acc: 0.8561 - val_loss: 0.2551 - val_acc: 0.9180
Epoch 3/10
70/70 [==============================] - 158s 2s/step - loss: 0.2676 - acc: 0.8863 - val_loss: 0.1784 - val_acc: 0.9491
Epoch 4/10
70/70 [==============================] - 156s 2s/step - loss: 0.2238 - acc: 0.9172 - val_loss: 0.1525 - val_acc: 0.9657
Epoch 5/10
70/70 [==============================] - 157s 2s/step - loss: 0.1723 - acc: 0.9364 - val_loss: 0.1295 - val_acc: 0.9741
Epoch 6/10
70/70 [==============================] - 156s 2s/step - loss: 0.1609 - acc: 0.9460 - val_loss: 0.0882 - val_acc: 0.9823
Epoch 7/10
70/70 [==============================] - 156s 2s/step - loss: 0.1448 - acc: 0.9504 - val_loss: 0.0794 - val_acc: 0.9751
Epoch 8/10
70/70 [==============================] - 156s 2s/step - loss: 0.1278 - a